In [1]:
import pandas as pd
from tqdm import tqdm
import re
import roman

In [2]:
checkliste = ['Kupfert',
              'Tit',
              'Titel',
              'Kupfertitel',
              'Titelbl',
              'Kupfertit',
              'Bildertit',
              'Frontisp',
              'Kupferfrontispiz',
              'Kupferfrontisp',
              'Porträtkupf',
              'Vort',
              'Portr.-Kupf.',
              'Frontispiz',
              'Front',
              'Port. Kupf.']

In [3]:
def seiten_ermitteln(umfang):
    seiten = 0
    fehlseiten = []
    merkliste = []
    try:
        for element in umfang.split(','):
            #print(element)
            if re_find := re.search('\[?(\d+)\]?\s.*\s?(Bl|Sp|S|Faltbl|Taf|Kupf|Blätter|Seiten|Spalten).*', element.strip()):
                    if re_find.group(2) == 'Bl' or re_find.group(2) == 'Faltbl' or re_find.group(2) == 'Taf' or re_find.group(2) == 'Blätter' or re_find.group(2) == 'Kupf':
                        seiten += int(re_find.group(1)) * 2
                        #print('finde bl')
                        if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl * 2
                            merkliste = []
                    elif re_find.group(2) == 'S' or re_find.group(2) == 's' or re_find.group(2) == 'Seiten':
                        #print('finde s')
                        seiten += int(re_find.group(1))
                        if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl
                            merkliste = []
                    elif re_find.group(2) == 'Sp' or re_find.group(2) == 'Spalten':
                        seiten += int(re_find.group(1))/4
                        #print(f'{re_find.group(1)} Spalten = {str(int(re_find.group(1))/4)} Seiten')
                    else:
                        None
            elif re_find := re.search('(^(?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3}))(\s(?P<typ>S|Bl|Seiten|Blätter|Taf|Tafeln\.))?.*$', element.strip()):
                    if re_find.group('typ')  == 'S' or re_find.group('typ') == 'Seiten':
                        seiten += roman.fromRoman(re_find.group(1).strip())
                        if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl
                            merkliste = []

                    elif re_find.group('typ')  == 'Bl' or re_find.group('typ') == 'Blätter' or re_find.group('typ')== 'Taf' or re_find.group('typ') == 'Tafeln':
                        seiten += roman.fromRoman(re_find.group(1).strip()) * 2
                        if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl * 2
                            merkliste = []
                    else:
                        merkliste.append(roman.fromRoman(re_find.group(1).strip()))
            elif re_find := re.search('(?P<typ>S\.|Seiten)\s?(?P<von>\d+)\s?-\s?(?P<bis>\d+)', element.strip()):
                    seiten += int(re_find.group('bis')) - int(re_find.group('von'))
                    if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl
                            merkliste = []
            elif re_find := re.search('Bl.\s?(\d+)\s?-\s?(\d+)', element.strip()):
                    seiten += (int(re_find.group(2)) - int(re_find.group(1))) * 2
                    if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl * 2
                            merkliste = []
            elif re_find := re.search('Sp.\s?(\d+)\s?-\s?(\d+)', element.strip()):
                    seiten += (int(re_find.group(2)) - int(re_find.group(1))) / 2
                    if len(merkliste) > 0:
                            for zahl in merkliste:
                                seiten += zahl / 4
                            merkliste = []
            elif any(i in element.strip() for i in checkliste):
                    seiten += 2
            elif re_find := re.match('\[?(\d+)\]?', element.strip()):
                    merkliste.append(int(re_find.group(1)))
            else:
                fehlseiten.append(element)
    except Exception as e:
        seiten = 0
        #print(e)
    return int(seiten)#, fehlseiten

In [4]:
buchformate = {'gr. 2': 45, '2': 40, 'gr. 4': 35, '4': 30, 'lex. 8': 25, 'gr. 8': 22.5, '8': 18.5, 'kl. 8': 15, '16': 10}

def groesse_ermitteln(format):
    if re_find := re.search(r'(\d*,?\d+)\s{1}cm', format):
        return float(re_find.group(1).replace(',','.'))
    elif re_find := re.search(r"((gr\.\s?|kl\.?\s?)?(\d{1,2}))°?", format):
        if re_find.group(1) in buchformate.keys():
            return buchformate[f'{re_find.group(1)}']
    else:
        return float("NaN")

In [15]:
df = pd.read_csv('buchbestand.csv')
df = df[df.year.str.len() == 4]
df['seiten'] = df.umfang.map(seiten_ermitteln, na_action='ignore')
df['groesse'] = df.format.map(groesse_ermitteln, na_action='ignore')
df.year = df.year.str.replace('x', '0', case=False)
df.verlag_ort = df.verlag_ort.str.replace('\[|\]','')
orte = pd.read_csv('dbsm_orte.csv')

In [16]:
pd.concat([df[pd.notna(df['herst_ort_name'])].merge(orte, how='left', left_on='herst_ort_name', right_on='ort'),df[pd.isna(df['herst_ort_name']) & pd.notna(df['verlag_ort'])].merge(orte, how='left', left_on='verlag_ort', right_on='ort')])



,idn,year,verlag_ort,herst_ort_idn,herst_ort_name,umfang,format,seiten,groesse,ort,location,lat,lon
0,200361929,1975,Berlin,040872955,Berlin,80 S.,24 cm,80.0,24.0,Berlin,"Berlin, 10117, Deutschland",52.517037,13.388860
1,200361929,1975,Berlin,040463346,Plauen,80 S.,24 cm,80.0,24.0,Plauen,"Plauen, Vogtlandkreis, Sachsen, Deutschland",50.495063,12.134652
2,203012658,1980,Praha,040754707,Nordmährisches Gebiet,"184, [52] S.",26 cm,236.0,26.0,NaN,NaN,NaN,NaN
3,203475739,1982,Köln,040314839,Köln,[4156] S.,17 cm,4156.0,17.0,Köln,"Köln, Nordrhein-Westfalen, Deutschland",50.938361,6.959974
4,204318025,1983,Berlin,040872955,Berlin,[26] S.,27 cm,26.0,27.0,Berlin,"Berlin, 10117, Deutschland",52.517037,13.388860
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102637,1233317245,2017,"Wolfville, NS",NaN,NaN,240 ungezählre Seiten,26 cm,240.0,26.0,"Wolfville, NS","Wolfville, Town of Wolfville, Kings County, No...",45.091407,-64.364510
102638,1233317911,2017,"Edmonton, Alberta, Canada",NaN,NaN,"xxxii, 321 Seiten",23 cm,321.0,23.0,"Edmonton, Alberta, Canada","Edmonton, Edmonton (city), Alberta, Canada",53.535411,-113.507996
102639,1233320459,2018,New York,NaN,NaN,1 Band (nicht paginiert),26 cm,0.0,26.0,New York,"New York, United States",40.712728,-74.006015
102640,123332232X,2017,Toronto,NaN,NaN,283 Seiten,26 cm,283.0,26.0,Toronto,"Toronto, Golden Horseshoe, Ontario, Canada",43.653482,-79.383935


In [23]:
df = pd.read_csv(f'buchbestand_plus.csv', usecols=['idn','year','verlag_ort','herst_ort_name','seiten','groesse'])
df['jahrzehnt'] = (10 * (df['year'] // 10)).astype(str)
df.groupby('jahrzehnt').median('seiten')

,year,seiten,groesse
jahrzehnt,,,
0,0,0.0,21.0
10,11,0.0,NaN
1000,1000,0.0,27.0
1150,1153,0.0,25.0
1380,1380,0.0,28.0
...,...,...,...
2000,2004,160.0,25.0
2010,2013,175.0,24.0
2020,2020,167.0,25.0


In [18]:
df = pd.read_csv('buchbestand.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 148134 entries, 0 to 148133
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   idn             148134 non-null  object
 1   year            145050 non-null  object
 2   verlag_ort      130178 non-null  object
 3   herst_ort_idn   25570 non-null   object
 4   herst_ort_name  25570 non-null   object
 5   umfang          139815 non-null  object
 6   format          125654 non-null  object
dtypes: object(7)
memory usage: 7.9+ MB
